##Импорт библиотек

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import copy
import time
from PIL import Image
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

##Импорт датасета с грибами

In [2]:
# https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/mNgIXBl4SxeNKg

In [3]:
! wget "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/mNgIXBl4SxeNKg" -O mushroom_archive.zip

--2021-10-27 11:09:58--  https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/mNgIXBl4SxeNKg
Resolving getfile.dokpub.com (getfile.dokpub.com)... 78.46.92.107
Connecting to getfile.dokpub.com (getfile.dokpub.com)|78.46.92.107|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader.disk.yandex.ru/disk/70c9ded39e3ec619cd1d2741b3aa59571b60c98f46d95e669d89d6ab12526f38/61796bc7/EnbAvYTdHMz2-Pr1wQCZdpaSsnguYEfYYi0fqSUGfWUabDEc82-MxWbelTjlQyfOrTKX1xW_BJZNXI7FHeWcnA%3D%3D?uid=0&filename=mushroom_archive.zip&disposition=attachment&hash=C57i7hkIKscZws9ohTEv1iicFTI/O8IP7CE1MYNryL6JGu%2BDpK9NUrxDj68404Jlq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=542830332&fsize=226515220&hid=d7f0776cf20a048bbfebef679293cbbc&media_type=compressed&tknv=v2 [following]
--2021-10-27 11:09:59--  https://downloader.disk.yandex.ru/disk/70c9ded39e3ec619cd1d2741b3aa59571b60c98f46d95e669d89d6ab12526f38/61796bc7/EnbAvYTdHMz2-Pr1wQCZ

In [4]:
!unzip mushroom_archive.zip -d mushroom_archive

Archive:  mushroom_archive.zip
   creating: mushroom_archive/test/
   creating: mushroom_archive/test/Белый гриб/
  inflating: mushroom_archive/test/Белый гриб/88.jpg  
  inflating: mushroom_archive/test/Белый гриб/92.jpg  
  inflating: mushroom_archive/test/Белый гриб/93.jpg  
  inflating: mushroom_archive/test/Белый гриб/95.jpg  
  inflating: mushroom_archive/test/Белый гриб/96.jpg  
  inflating: mushroom_archive/test/Белый гриб/98.jpg  
  inflating: mushroom_archive/test/Белый гриб/99.jpg  
   creating: mushroom_archive/test/Бледная поганка/
  inflating: mushroom_archive/test/Бледная поганка/86.jpg  
  inflating: mushroom_archive/test/Бледная поганка/87.jpg  
  inflating: mushroom_archive/test/Бледная поганка/88.jpg  
  inflating: mushroom_archive/test/Бледная поганка/89.jpg  
  inflating: mushroom_archive/test/Бледная поганка/90.jpg  
  inflating: mushroom_archive/test/Бледная поганка/91.jpg  
  inflating: mushroom_archive/test/Бледная поганка/92.jpg  
  inflating: mushroom_archive

In [5]:
! ls

mushroom_archive  mushroom_archive.zip	sample_data


In [6]:
! ls mushroom_archive

test  train  valid


In [7]:
! ls mushroom_archive/train

'Белый гриб'	    Груздь	     Масленок   Опенок	       Сморчок
'Бледная поганка'  'Лисичка гриб'    Моховик    Подберезовик   Сыроежка
 Волнушка	   'Ложный опенок'   Мухомор    Подосиновик


##Импорт предобученной Resnet18

In [8]:
model = models.resnet18(pretrained=True) # подключаем предобученную модель
model.eval() # метод eval вызывается, чтобы модель не начала заново обучаться, а только давала предсказания
model

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# приводим все изображения к единому виду с помощью фиксированных преобразований
resnet_transforms = transforms.Compose([
        transforms.Resize(256), # размер каждой картинки будет приведен к 256*256
        transforms.CenterCrop(224), # у картинки будет вырезан центральный кусок размера 224*224
        transforms.ToTensor(), # картинка из питоновского массива переводится в формат torch.Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # значения пикселей картинки нормализуются
    ])

##Создание выборок в формате PyTorch-датасета

In [10]:
train_data = datasets.ImageFolder('./mushroom_archive/train', transform=resnet_transforms)
val_data = datasets.ImageFolder('./mushroom_archive/valid', transform=resnet_transforms)
test_data = datasets.ImageFolder('./mushroom_archive/test', transform=resnet_transforms)

In [11]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

##Дообучение сети на новом датасете

###Заморозка сверточных слоев и замена последнего слоя

In [12]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [13]:
neurons = 14
model.fc = nn.Linear(512, neurons)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
len(list(model.children()))

10

In [15]:
layers_to_freeze = 8
for i, layer in enumerate(model.children()):
    if i < layers_to_freeze + 1:
        for param in layer.parameters():
            param.requires_grad = False

In [16]:
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


###Обучение сети

In [17]:
def train(model, n_epoch=6):
    # выбираем функцию потерь
    loss_fn = torch.nn.CrossEntropyLoss()

    # выбираем алгоритм оптимизации и learning_rate
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_accuracies = []
    val_losses = []
    
    # обучаем сеть
    for epoch in tqdm_notebook(range(n_epoch)):
        train_dataiter = iter(train_loader)
        running_loss = 0.0

        model.train(True)
        for i, batch in enumerate(tqdm_notebook(train_dataiter)):
            X_batch, y_batch = batch # так получаем текущий батч
            
            logits = model(X_batch) # forward pass (получение ответов на батч картинок)
            loss = loss_fn(logits, y_batch) # вычисление лосса от выданных сетью ответов и правильных ответов на батч
            loss.backward() # backpropagation (вычисление градиентов)
            optimizer.step() # обновление весов сети
            optimizer.zero_grad() # обнуляем веса
            # выведем текущий loss
            running_loss += loss.item()

            # выведем лосс каждые 50 батчей
            if i % 50 == 49:
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 49))
                train_losses.append(running_loss / 49)
                running_loss = 0.0


        

        # evaluate on val
        model.train(False)
        val_dataiter = iter(val_loader)


        val_loss_per_epoch = 0
        val_accuracy_per_epoch = 0
        for i, batch in enumerate(tqdm_notebook(val_dataiter)):
            # так получаем текущий батч
            X_batch, y_batch = batch
            with torch.no_grad():
                logits = model(X_batch)
                y_pred = torch.argmax(logits, dim=1)
                val_accuracy_per_epoch += np.mean(y_pred.numpy() == y_batch.numpy())

                val_loss_per_epoch += loss_fn(logits, y_batch)

        val_accuracies.append(val_accuracy_per_epoch / (i + 1))
        val_losses.append(val_loss_per_epoch / (i + 1))

    print('Обучение закончено')
    return model, train_losses, val_losses, val_accuracies

In [18]:
model, train_losses, val_losses, val_accuracies = train(model, n_epoch=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/14 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Обучение закончено


##Проверка метрики на тестовой выборке

In [19]:
def evaluate(model, dataloader, name='test'):
    accuracy = 0.

    dataiter = iter(dataloader)
    for i, batch in enumerate(tqdm_notebook(dataiter)):
        # так получаем текущий батч
        X_batch, y_batch = batch
        with torch.no_grad():
            logits = model(X_batch)
            y_pred = torch.argmax(logits, dim=1)
            accuracy += np.mean(y_pred.numpy() == y_batch.numpy())
    print(f'{name} accuracy is {accuracy / (i + 1)}')
    return accuracy / (i + 1)

In [24]:
test_dataiter = iter(test_loader)
evaluate(model, test_dataiter, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3 [00:00<?, ?it/s]

test accuracy is 0.6736709770114943


0.6736709770114943

## Сохранение модели

In [21]:
torch.save(model, 'mushroom_model.pt')

##Загрузка модели из файла

In [22]:
model_new = torch.load('mushroom_model.pt')

In [23]:
test_dataiter = iter(test_loader)
evaluate(model_new, test_dataiter, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3 [00:00<?, ?it/s]

test accuracy is 0.6736709770114943


0.6736709770114943

In [25]:
!pip install pipreqs

In [26]:
!pipreqs /content/

INFO: Successfully saved requirements file in /content/requirements.txt
